In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 


def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))



customer_id = '32'
customer_name='%eddiebauer%'
client='EB'

In [2]:
dateszs='2001-08-11'
attribut='features'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
461758


# Check for Attribute bucket Values

In [3]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['color']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

54
Number of attributes: 54

Start Buckets
0 Attribute: bag_style
Number of SKUs for the attribute bag_style: 188
Start Buckets
1 Attribute: bed_size
Number of SKUs for the attribute bed_size: 36
Start Buckets
2 Attribute: bedding_size
Number of SKUs for the attribute bedding_size: 53
Start Buckets
3 Attribute: capacity_by_ounce
Number of SKUs for the attribute capacity_by_ounce: 49
Start Buckets
4 Attribute: care
Number of SKUs for the attribute care: 32109
Start Buckets
5 Attribute: closure
Number of SKUs for the attribute closure: 37767
Start Buckets
6 Attribute: collar_style
Number of SKUs for the attribute collar_style: 0
Start Buckets
7 Attribute: color
Number of SKUs for the attribute color: 41310
Start Buckets
8 Attribute: drinkware_type
Number of SKUs for the attribute drinkware_type: 67
Start Buckets
9 Attribute: fabric
Number of SKUs for the attribute fabric: 38466
Start Buckets
10 Attribute: features
Number of SKUs for the attribute features: 38104
Buckets Effected: 1
Numbe

# Check Freetext Attributes

In [ ]:
# patterns = {
#     'appliance_cubic_feet': r'(cu ft)|(n\/a)|()',
# }

In [ ]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

# Curated

# Uncurated

In [ ]:
# parameters
formatted_attribute = 'sustainability'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

# send to the folder for upload

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good(client, attribute,matchesna,today)   

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
    
looks_good(client, attribute,matchesdiam,today)  